# Multi-Armed Bandit Problem for Article Recommendation

In this notebook, we solve a problem that simulates article recommendation optimization using a **Multi-Armed Bandit (MAB)** approach. The goal is to recommend articles to maximize the total views (or clicks), with some articles being politically or commercially aligned with the platform’s goals. We use two strategies for solving the problem: **Epsilon-Greedy** and **Thompson Sampling**.

## Problem Overview

In this setup, each article can be thought of as an "arm" in the bandit problem, where the goal is to select which article to recommend to users. Each article has an associated reward, which corresponds to the number of views or clicks the article generates. Our task is to maximize the total views for politically or commercially aligned articles over time.

The challenge is that we do not know the true rewards of each article at the beginning, and our objective is to learn which articles yield the highest rewards through exploration and exploitation.

## Multi-Armed Bandit Solution

We simulate two strategies to solve this problem:

1. **Epsilon-Greedy Strategy**:
    - This strategy balances exploration (choosing random articles) with exploitation (choosing the article that has the highest estimated reward). 
    - With probability epsilon (`0.1` in this case), it explores a random article, while with probability `1 - epsilon`, it selects the article that has the highest estimated reward.

2. **Thompson Sampling Strategy**:
    - Thompson Sampling is a Bayesian method that probabilistically selects an article based on its expected reward. 
    - It maintains a belief about the reward distribution for each article and uses a Beta distribution to model the likelihood of an article being the best choice.
    - After each selection, it updates its belief based on the observed reward.


In [1]:
import numpy as np
import random

class KArmBandit:
    def __init__(self, k, epsilon=0.1):
        self.k = k
        self.epsilon = epsilon
        self.q_true = np.random.normal(0, 1, k)
        self.q_est = np.zeros(k)
        self.n = np.zeros(k)
        print(f"True rewards for each article: {self.q_true}")

    def select_arm(self):
        if random.random() < self.epsilon:
            arm = random.randint(0, self.k - 1)
            print(f"Exploring: Selected article {arm}")
        else:
            arm = np.argmax(self.q_est)
            print(f"Exploiting: Selected article {arm} with estimated reward {self.q_est[arm]:.2f}")
        return arm
    
    def update(self, arm, reward):
        self.n[arm] += 1
        self.q_est[arm] += (reward - self.q_est[arm]) / self.n[arm]

    def simulate(self, timesteps=1000):
        total_reward = 0
        for t in range(timesteps):
            arm = self.select_arm()
            reward = np.random.normal(self.q_true[arm], 1)
            self.update(arm, reward)
            total_reward += reward
            
            if t % 100 == 0:
                print(f"Step {t}: Selected article {arm}, Reward: {reward:.2f}, Total reward: {total_reward:.2f}")
                
        return total_reward


In [2]:
class ThompsonSamplingBandit:
    def __init__(self, k):
        self.k = k
        self.q_true = np.random.normal(0, 1, k)
        self.alpha = np.ones(k)
        self.beta = np.ones(k)
        print(f"True rewards for each article: {self.q_true}")

    def select_arm(self):
        theta_samples = np.random.beta(self.alpha, self.beta)
        arm = np.argmax(theta_samples)
        print(f"Selected article {arm} with Thompson Sampling, sampled reward mean: {theta_samples[arm]:.2f}")
        return arm

    def update(self, arm, reward):
        if reward > 0:
            self.alpha[arm] += 1
        else:
            self.beta[arm] += 1

    def simulate(self, timesteps=1000):
        total_reward = 0
        for t in range(timesteps):
            arm = self.select_arm()
            reward = np.random.normal(self.q_true[arm], 1)
            self.update(arm, reward)
            total_reward += reward
            
            if t % 100 == 0:
                print(f"Step {t}: Selected article {arm}, Reward: {reward:.2f}, Total reward: {total_reward:.2f}")
                
        return total_reward

In [3]:
k = 5
timesteps = 1000

bandit_epsilon = KArmBandit(k, epsilon=0.1)
bandit_thompson = ThompsonSamplingBandit(k)

print("\nRunning Epsilon-Greedy Simulation:")
total_reward_epsilon = bandit_epsilon.simulate(timesteps)

print("\nRunning Thompson Sampling Simulation:")
total_reward_thompson = bandit_thompson.simulate(timesteps)

True rewards for each article: [0.19242152 1.31776345 0.57047497 1.30514983 0.45315534]
True rewards for each article: [-0.25483373  0.92134728  0.8979083   1.62813552 -1.65736917]

Running Epsilon-Greedy Simulation:
Exploiting: Selected article 0 with estimated reward 0.00
Step 0: Selected article 0, Reward: 0.22, Total reward: 0.22
Exploiting: Selected article 0 with estimated reward 0.22
Exploiting: Selected article 1 with estimated reward 0.00
Exploring: Selected article 2
Exploiting: Selected article 1 with estimated reward 1.16
Exploiting: Selected article 1 with estimated reward 1.81
Exploiting: Selected article 1 with estimated reward 1.88
Exploiting: Selected article 1 with estimated reward 1.71
Exploiting: Selected article 1 with estimated reward 1.54
Exploiting: Selected article 1 with estimated reward 1.46
Exploiting: Selected article 1 with estimated reward 1.50
Exploiting: Selected article 1 with estimated reward 1.59
Exploiting: Selected article 1 with estimated reward 1

In [4]:
print(f"\nTotal reward using Epsilon-Greedy: {total_reward_epsilon:.2f}")
print(f"Total reward using Thompson Sampling: {total_reward_thompson:.2f}")


Total reward using Epsilon-Greedy: 1188.72
Total reward using Thompson Sampling: 1466.61


## Results

Both strategies are simulated over 1000 timesteps, where each timestep represents a new decision on which article to recommend. After 1000 timesteps, the total reward (views) accumulated by each strategy is printed.

- **Epsilon-Greedy**: The reward here reflects the results of balancing exploration and exploitation using the epsilon-greedy strategy.
- **Thompson Sampling**: The reward reflects the results of probabilistically selecting articles based on Thompson Sampling.

By comparing the total rewards of both strategies, we can evaluate which method is more effective in maximizing views in a dynamic and uncertain environment.

## Conclusion

This simulation highlights how different strategies for balancing exploration and exploitation can be applied to article recommendation systems. In real-world scenarios, these algorithms can help optimize content recommendations based on various factors such as user engagement, content alignment with political/commercial interests, and more.
